# Preparing CNES to ETL process

This notebook aims to generate an intermediate CNES file to feed the ETL process. CNES will be grouped by State/year/month of the beginning of the contract, and transformed in a way that they can become inputs of the ETL process.

In [1]:
import os
import pandas as pd

## 1- Global variables

Absolute paths and file names, both original and export.

In [2]:
path_abs = 'C:/Users/hbrac/OneDrive/Documentos/GEESC/OSS/Fase 2/Data/Transformed data'
file_name = 'base_unificada_padronizada.xlsx'
file_export = 'ano_mes_cnes.xlsx'

## 2- Data loading and visualization

In [3]:
df = pd.read_excel(os.path.join(path_abs, file_name), converters = {'cnes': str, 'ano_vigencia': str})
df.head()

,uf,municipio,cnes,nome_estabelecimento,tipo_estabelecimento,gestao,cnpj,razao_social,tipo_entidade,forma_contratacao,...,inicio_vigencia,termino_vigencia,ano_vigencia,valor_contrato,recurso_federal,recurso_estadual,recurso_municipal,fonte,origem,razao_social_padr
0,35,355410,9128743,SAMU 192 TAUBATE BASE PIRACANGAGUA AVANCADA,997,1,99999999999999,SEM NOME REGISTRADO,1,1,...,NaN,NaN,2016,999999999.0,9,9,9,9,0,SEM NOME REGISTRADO
1,35,354230,9133232,SAMU 192 TAUBATE BASE REDENCAO DA SERRA,997,1,99999999999999,SEM NOME REGISTRADO,1,1,...,NaN,NaN,2016,999999999.0,9,9,9,9,0,SEM NOME REGISTRADO
2,21,210000,6872174,CENTRO DE MEDICINA ESPECIALIZADA CEMESP,16,2,99999999999999,SEM NOME REGISTRADO,1,1,...,NaN,NaN,9999,999999999.0,9,9,9,9,0,SEM NOME REGISTRADO
3,21,210350,2726645,HOSPITAL REGIONAL DR CARLOS MACIEIRA,6,1,99999999999999,SEM NOME REGISTRADO,1,1,...,NaN,NaN,9999,999999999.0,9,9,9,9,3,SEM NOME REGISTRADO
4,42,421660,2663651,CENTRO DE SAUDE ROCADO,1,1,99999999999999,SEM NOME REGISTRADO,1,1,...,NaN,NaN,9999,999999999.0,9,9,9,9,0,SEM NOME REGISTRADO


## 3- Transformation

The main purpose of this notebbok. The process will be conducted in the following steps.

### 3.1- Data filtering

Data will be filtered, removing repeated entries.

In [4]:
df = df[~((df.origem == 3)&(pd.isna(df.inicio_vigencia)))]
df.ano_vigencia[df.ano_vigencia.astype(float)<2006]='9999'
df = df.sort_values(by = ['ano_vigencia'], ascending = True)
df = df.drop_duplicates(subset=['cnes'], keep = 'last').reset_index(drop = True)

### 3.2- Column dropping

Columns that aren't pertinent to this process will be dropped.

In [5]:
df.drop(columns = ['municipio', 'nome_estabelecimento', 'tipo_estabelecimento', 'gestao', 'cnpj', 'razao_social', 'tipo_entidade', 'forma_contratacao', 'termino_vigencia', 'valor_contrato', 'recurso_federal', 'recurso_estadual', 'recurso_municipal', 'fonte', 'origem', 'numero_contrato', 'razao_social_padr'], inplace = True)
df.head()

,uf,cnes,inicio_vigencia,ano_vigencia
0,29,2487438,NaN,2006
1,29,4028511,NaN,2006
2,35,2077477,NaN,2007
3,35,5420938,NaN,2007
4,35,2076896,NaN,2008


### 3.3- Column creation and treatment of missing values

A column named 'mes_vigencia' will be created, extracting the month from the (date) field 'inicio_vigencia'. When this information is missing, June will be inputed as the starter month. If year is also absent, then the date will be April, 2025.

In [6]:
df['mes_vigencia'] = ''

for i in range(len(df)):
    if pd.isna(df.loc[i, 'inicio_vigencia']):
        df.loc[i, 'mes_vigencia'] = '06'
    else:
        df.loc[i, 'mes_vigencia'] = df.loc[i, 'inicio_vigencia'].split('-')[1]

df.mes_vigencia[df.ano_vigencia == '9999'] = '04'        
df.ano_vigencia[df.ano_vigencia == '9999'] = '2025'   
df.head()

C:\Users\hbrac\AppData\Local\Temp\ipykernel_1564\458556170.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mes_vigencia[df.ano_vigencia == '9999'] = '04'
C:\Users\hbrac\AppData\Local\Temp\ipykernel_1564\458556170.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ano_vigencia[df.ano_vigencia == '9999'] = '2025'


,uf,cnes,inicio_vigencia,ano_vigencia,mes_vigencia
0,29,2487438,NaN,2006,06
1,29,4028511,NaN,2006,06
2,35,2077477,NaN,2007,06
3,35,5420938,NaN,2007,06
4,35,2076896,NaN,2008,06


### 3.4- Recoding State data

The State data will be recoded to its abbreviation, since this format is used by DataSUS.

In [7]:
df.uf = df.uf.astype(str).replace('12', 'AC')
df.uf = df.uf.astype(str).replace('27', 'AL')
df.uf = df.uf.astype(str).replace('16', 'AP')
df.uf = df.uf.astype(str).replace('13', 'AM')
df.uf = df.uf.astype(str).replace('29', 'BA')
df.uf = df.uf.astype(str).replace('23', 'CE')
df.uf = df.uf.astype(str).replace('53', 'DF')
df.uf = df.uf.astype(str).replace('32', 'ES')
df.uf = df.uf.astype(str).replace('52', 'GO')
df.uf = df.uf.astype(str).replace('21', 'MA')
df.uf = df.uf.astype(str).replace('51', 'MT')
df.uf = df.uf.astype(str).replace('50', 'MS')
df.uf = df.uf.astype(str).replace('31', 'MG')
df.uf = df.uf.astype(str).replace('15', 'PA')
df.uf = df.uf.astype(str).replace('25', 'PB')
df.uf = df.uf.astype(str).replace('41', 'PR')
df.uf = df.uf.astype(str).replace('26', 'PE')
df.uf = df.uf.astype(str).replace('22', 'PI')
df.uf = df.uf.astype(str).replace('24', 'RN')
df.uf = df.uf.astype(str).replace('43', 'RS')
df.uf = df.uf.astype(str).replace('33', 'RJ')
df.uf = df.uf.astype(str).replace('11', 'RO')
df.uf = df.uf.astype(str).replace('14', 'RR')
df.uf = df.uf.astype(str).replace('42', 'SC')
df.uf = df.uf.astype(str).replace('35', 'SP')
df.uf = df.uf.astype(str).replace('28', 'SE')
df.uf = df.uf.astype(str).replace('17', 'TO')


df.drop(columns = ["inicio_vigencia"], inplace = True)
df.head()

,uf,cnes,ano_vigencia,mes_vigencia
0,BA,2487438,2006,06
1,BA,4028511,2006,06
2,SP,2077477,2007,06
3,SP,5420938,2007,06
4,SP,2076896,2008,06


### 3.5- Adding quotation marks to columns

To conform to the ELT syntax, all the columns will be between quotes.

In [8]:
df.cnes = '"' + df.cnes + '"'
df.head()

,uf,cnes,ano_vigencia,mes_vigencia
0,BA,"""2487438""",2006,06
1,BA,"""4028511""",2006,06
2,SP,"""2077477""",2007,06
3,SP,"""5420938""",2007,06
4,SP,"""2076896""",2008,06


### 3.6- Grouping CNES

CNES data will be grouped by State/year/month and transformed into a string separeted by commas.

In [9]:
df_grouped = df.groupby(['uf', 'ano_vigencia', 'mes_vigencia'])['cnes'] \
               .apply(lambda x: ', '.join(x)).reset_index()
df_grouped.cnes = "[" + df_grouped.cnes + "]"
df_grouped.head()

,uf,ano_vigencia,mes_vigencia,cnes
0,AC,2021,01,"[""5336171""]"
1,AL,2017,09,"[""5616298""]"
2,AL,2018,06,"[""2720043""]"
3,AL,2019,06,"[""2008750"", ""2008785""]"
4,AL,2019,10,"[""4156730""]"


## 4 - Export

Data will be exported as an Excel workbook.

In [10]:
df_grouped.to_excel(os.path.join(path_abs, file_export), index = False)